In [1]:
import argparse
import struct
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad

#Define constants
CHUNK_SIZE = 1000
BLOCK_SIZE = 16

#     # Load firmware binary from infile
#     with open(infile, 'rb') as fp:
#         firmware = fp.read()

#Load test_firmware Binary
with open("test_firmware.bin", 'rb') as fp:
    firmware = fp.read()
    
message = "YEEEEEEEEEEEEEEEEEEEEET"

version = 3

print(len(firmware))

# Create variable to hold message
msg = message.encode() + b'\00'

# Pack version and size into two little-endian shorts
# metadata = struct.pack('<HH', version, len(firmware))

#Split firmware into 1K chunks

chunks_needed = int(len(firmware)/CHUNK_SIZE)

chunks=list()

for i in range(chunks_needed):
    print(i)
    chunks.append(firmware[i * CHUNK_SIZE:(i + 1) * CHUNK_SIZE])

if(CHUNK_SIZE * (chunks_needed) - len(firmware) != 0):
    chunks.append(firmware[CHUNK_SIZE * (chunks_needed):])

#Encrypt each chunk

#Initialize AES

key = get_random_bytes(16) #For Testing Purposes only

#Encrypt each Chunk with AES

final_output = b'' #Final output?


for i, chunk in enumerate(chunks):
    
    #Set up AES Cipher
    aes_cipher = AES.new(key, AES.MODE_GCM)
    #Set up metadata
    metadata = struct.pack('<hhhh', len(pad(chunk, BLOCK_SIZE)), version, len(firmware), i)
    aes_cipher.update(metadata)
    
    #Get Cipher Text
    ciphertext, tag = aes_cipher.encrypt_and_digest(pad(chunk, BLOCK_SIZE))
    
    #THings for testing :D
#     print(metadata)
#     print(ciphertext)
#     print(len(ciphertext))
#     print(tag)
#     print(len(tag))

    #Add result to final output
    final_output += (metadata + aes_cipher.nonce + tag + ciphertext)

# Add release message

aes_cipher = AES.new(key, AES.MODE_GCM)
#Set up metadata
metadata = struct.pack('<hhhh', len(pad(msg, BLOCK_SIZE)), version, len(firmware), -1)
aes_cipher.update(metadata)
#Get Cipher Text
ciphertext, tag = aes_cipher.encrypt_and_digest(pad(chunk, BLOCK_SIZE))
final_output += (metadata + aes_cipher.nonce + tag + ciphertext)

#More Testing
print(final_output)

# Write firmware blob to outfile
# with open(outfile, 'wb+') as outfile:
#     outfile.write(firmware_blob)


6541
0
1
2
3
4
5
b'\xf0\x03\x03\x00\x8d\x19\x00\x00#,&\xfa\xd2[\xc6i\x86\xfeM\xe06\x11\xb2\x00\x12\xc5\x9b\x13^\xa5vg\xd2\x00S\x8c[\x10\x15\xee]e\xf2\xd1\x97\xe2\xf7kJ|\x1a\\\xf7\xf2\x1c\x8d\xa8pn\xeb*vv^Y\xa5\nB\x16M1&\xf9\xbb\x9ea\xd4\xe9*?X\x7fr\x93#\xa6\xbe\\\xe0\x03\xc6\xea\x92\xb7c\x1a\xec\x0c\xb6\x0e_\x92\xe8\xeb\x10\x80\xbf;8\x03Z<\xa6wt=\xab\x1bF\xb7m\x87\x86d\xc5\xc4h\x1e\x06j\xc0p\xc0I\xfe\x889\xeeqLn\x8e\xe1G\xe9GI\xb0\xcd\xfd\xb5\xf3da\xbe\x85+u\xc5\x94\xe5\'\xb0\xac\x04\x99\x1e\x97\t\xf8~p\xfb\xc1\x94`<\xfb\x8dHL\xe6\xc5~\xe8p6D\x1dn\x8a\x8dG;:1j\xe1\x06Y\xff\x94\xaf\x00\x10su\x93\xe5\xe9y\xf4\x0e8\nI\xbc\xfd\x9a\x19\xc2WV<\xec<\xe3\x1c\xe22\x8dh\x08\xd6($\x11\x0eZ:\xa8\xdfw\x8ct\x18\x83\x05p\x0f!\x80\xeb\x11\x1b\xdd\xb0\xedj ]\x99\xa3\x1e\'[@D\xde\xfc\xed#\xe0D\x853\x07\xaa\xe6\x02\x97\xf8\xe9\xad\xd53\x19R#\xb6;\xe9\xa0F\xa0\x1e\xbb\xee\xd8u\xe7ZwJ\x0cMB\x11\x81\xc8\x98X\xa2f#\x8e\x1d\xc2\xbdT\xda\xcb\xe6\xcb\xac\xbb\\\xf6\xa2\x9a|\xfbE>Q\xe2Z\xd2\xd2\xed\xe7\xe7ha\xfe\

In [3]:
import argparse
import struct
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA

#Define constants
CHUNK_SIZE = 1000
BLOCK_SIZE = 16

#     # Load firmware binary from infile
#     with open(infile, 'rb') as fp:
#         firmware = fp.read()

#Load test_firmware Binary
with open("test_firmware.bin", 'rb') as fp:
    firmware = fp.read()
    
message = "YEEEEEEEEEEEEEEEEEEEEET"

version = 3

print(len(firmware))

with open('secret_build_output.txt', 'rb') as fp:
        secrets = fp.read()

    key = secrets[0:16]
#     rsa_key = RSA.import_key(secrets[16:])

# Create variable to hold message
msg = message.encode() + b'\00'

# Pack version and size into two little-endian shorts
# metadata = struct.pack('<HH', version, len(firmware))

#Split firmware into 1K chunks

chunks_needed = int(len(firmware)/CHUNK_SIZE)

chunks=list()

for i in range(chunks_needed):
    print(i)
    chunks.append(firmware[i * CHUNK_SIZE:(i + 1) * CHUNK_SIZE])

if(CHUNK_SIZE * (chunks_needed) - len(firmware) != 0):
    chunks.append(firmware[CHUNK_SIZE * (chunks_needed):])

#Encrypt each chunk

#Initialize AES & RSA Keys for testing. 

key = get_random_bytes(16) #For Testing Purposes only
rsa_key = RSA.generate(2048) #For Testing Purposes only


#Encrypt each Chunk with AES

final_output = b'' #Final output?


for i, chunk in enumerate(chunks):
    
    #Set up AES Cipher
    aes_cipher = AES.new(key, AES.MODE_GCM)
    #Set up metadata
    metadata = struct.pack('<hhhh', len(pad(chunk, BLOCK_SIZE)), version, len(firmware), i)
    aes_cipher.update(metadata)
    
    #Get Cipher Text
    ciphertext, tag = aes_cipher.encrypt_and_digest(pad(chunk, BLOCK_SIZE))
    
    h = SHA256.new(ciphertext)
    signature = pkcs1_15.new(rsa_key).sign(h)
    
    #THings for testing :D
#     print(metadata)
#     print(ciphertext)
#     print(len(ciphertext))
#     print(tag)
#     print(len(tag))
    print(signature)
    print(len(signature))

    #Add result to final output
    final_output += (signature + metadata + aes_cipher.nonce + tag + ciphertext)

# Add release message

aes_cipher = AES.new(key, AES.MODE_GCM)
#Set up metadata
metadata = struct.pack('<hhhh', len(pad(msg, BLOCK_SIZE)), version, len(firmware), -1)
aes_cipher.update(metadata)
#Get Cipher Text
ciphertext, tag = aes_cipher.encrypt_and_digest(pad(chunk, BLOCK_SIZE))
final_output += (metadata + aes_cipher.nonce + tag + ciphertext)

#More Testing
print(final_output)
print(len(final_output))

# Write firmware blob to outfile
# with open(outfile, 'wb+') as outfile:
#     outfile.write(firmware_blob)


2826
0
1
Private RSA key at 0x7F78E86DBE10
b'\x077\xce\xfc\xa5\xd8@\xc1%\xde\x06\xcb\x0e-\x1b\x81:P\xbf\xa7%\x85z\x91\xebM\x9d\x15\x83l\xb4\xa0\xf8\xb6\xea\xb8?\xe3\t\xcaRc\xc1mJ\xe1\x04\x90\xdb\x90\x90\t(\x1es\xb5\x08z9\x0c\x8cP\x15\xc8\xa8\x98"\xc1\x1e\x87\x80\xe1%\x9d\x0f\xb3\xa8\n@\x0e\xe7\xad\xf9]/\t%\x8f- \xf3\xae\xf8\xc1\xbe@\xcdB"\xfad4j\xc7_Nb*\xc2\x1b\xf1\xbd\xe3\x06\xfb)T\xb7-\x81\xbaJ\x90\xc5\x8eq\xf34\x17e\xfcR*1x\xc2\xe6\xbf\x13\xd9v\xed\x84}n@\xd9\xca\xa6"\x16\xf2\xaf#$\xc7\x18\xc3\x1c\xf3\x97K\x93\xa2\xc2W\xe3a(c\x02\xaf\n\xa1\x9bbC\x01\xf7MgW\xf7L<GD\xc0\x07\xde\xd3\x1d9\r\xdf`!@t{\\Qy\xe9\x9b\x03W\xb7Qd\x05]p\xc3\xd3\xe4\x0c\xe6\x03"#\xdb\x85\xed^\x7fj\x93\xd6\x006\xd1~\x80\xe9g\x99:\xb4\xcb\xb2\x90L\xc7BN\x8b\xd1_\xc3\xe0N\x1c%\x8d-'
256
b"\x0e\xb0z,}(\x1a\xa0\x82^\xc8\xaf\xbb\xa04\xd9\x89\x1c2\xe0\xcap'P\xad\x86\xee\x9fve\xea\xbb=\x14\x9bC\xaf9U$]\x92\x02\x1b\xf5\\XF\x8f\x8ap\xa8\xd9\xb7\x8d\x1c\xb4\x06.?\xef\x8c!:\x95\x17 \t=\xb22\x92\xb4\x13;\x89$V\xf2\xef\xeb\xcf

In [2]:
from Crypto.PublicKey import RSA
rsa_key = RSA.generate(2048)

print(rsa_key.d)

720137342253934599311419425089242706997723288364452349443481562873979756588296848075209925874209311149158902090346873264621138808749873508162883339000327479232628308425010712468433476044169517305279278775719991863065875925922347551064652357150969439914472619095681651631883604506920323325259372518158615857577767058309425354510269109297588635763643393307222841758543168519255540519488882321258612280102502138192328684859562084883010525111408681914665344431636710493093470709119237900267771792186945100560433311149922503582274825877506731373473882494783682440202622765666050078115143068303396225927618797737378800177


## AES Test File


In [ ]:
"""
Firmware Bundle-and-Protect Tool

"""
import argparse
import struct
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA


def protect_firmware(infile, outfile, version, message):

    

    #Define constants
    CHUNK_SIZE = 1000
    BLOCK_SIZE = 16

#     # Load firmware binary from infile
#     with open(infile, 'rb') as fp:
#         firmware = fp.read()

    #TEST: load in known bin file
    with open('test_firmware.bin', 'rb') as fp:
        firmware = fp.read()
        
    #Load in Keys
    
    with open('secret_build_output.txt', 'rb') as fp:
        secrets = fp.read()

    key = secrets[0:16]
    
    print(key)
    
#     FORRRRR LAAAAAAAAAAATEEEEEEEEEEEEEEEEEERRRRRRRRRRRRR
#     rsa_key = RSA.import_key(secrets[16:])
    
#     print(rsa_key)
    
#     Print keys. 
    
    
    
    
    # Create variable to hold message
    msg = message.encode() + b'\00'
    
    print("MSG")
    print(msg)

    #Split firmware into 1K chunks

    chunks_needed = int(len(firmware)/CHUNK_SIZE)
    
    print("CHUNKS_NEEDED")
    print(chunks_needed)
    print(len(firmware))

    chunks=list()

    for i in range(chunks_needed):
        print(i)
        chunks.append(firmware[i * CHUNK_SIZE:(i + 1) * CHUNK_SIZE])

    if(CHUNK_SIZE * (chunks_needed) - len(firmware) != 0):
        chunks.append(firmware[CHUNK_SIZE * (chunks_needed):])

#     #Encrypt each chunk

#     #Initialize AES & RSA Keys for testing. 

#     key = get_random_bytes(16) #For Testing Purposes only
#     rsa_key = RSA.generate(2048) #For Testing Purposes only


#     #Encrypt each Chunk with AES

    final_output = b'' #Final output?


    for i, chunk in enumerate(chunks):

        #Set up AES Cipher
        aes_cipher = AES.new(key, AES.MODE_GCM)
        #Set up metadata
        metadata = struct.pack('<hhhh', len(pad(chunk, BLOCK_SIZE)), version, len(firmware), i)
        aes_cipher.update(metadata)

        #Get Cipher Text
        ciphertext, tag = aes_cipher.encrypt_and_digest(pad(chunk, BLOCK_SIZE))
        nonce = aes_cipher.nonce

#         h = SHA256.new(ciphertext)
#         signature = pkcs1_15.new(rsa_key).sign(h)

        #THings for testing :D
    #     print(metadata)
        print(i)
#         print(ciphertext)
        print(len(chunk))
        print(len(ciphertext))
        print(nonce)
    #     print(tag)
    #     print(len(tag))
#         print(signature)
#         print(len(signature))

        #Add result to final output
        #NON_RSA
        print("TAG")
        print(tag)
        final_output += (metadata + nonce + tag + ciphertext)
        
        plain = decrypt(nonce, metadata, ciphertext, tag, key)
        print(plain)
        #RSA
#         final_output += (signature + metadata + aes_cipher.nonce + tag + ciphertext)

#     # Add release message

#     aes_cipher = AES.new(key, AES.MODE_GCM)
#     #Set up metadata
#     metadata = struct.pack('<hhhh', len(pad(msg, BLOCK_SIZE)), version, len(firmware), -1)
#     aes_cipher.update(metadata)
#     #Get Cipher Text
#     ciphertext, tag = aes_cipher.encrypt_and_digest(pad(chunk, BLOCK_SIZE))
#     final_output += (metadata + aes_cipher.nonce + tag + ciphertext)

#     #More Testing

    for i in range(6):
        start = (i * 1048)
        end = start + 1048
        all_data = final_output[start:end]
        print(all_data)
        metadata = all_data[0:8]
        print(metadata)
        nonce = all_data[8:24]
        print(nonce)
        tag = all_data[24:40]
        print("TAG")
        print(tag)
        ciphertext = all_data[40:]
        decrypt(nonce, metadata, ciphertext, tag, key)
        
        
#     print(final_output)
#     print(len(final_output))

#     #     Write firmware blob to outfile
#     with open(outfile, 'wb+') as outfile:
#         outfile.write()

def decrypt(nonce_var, metadata, cipher_text, tag_var, key):
    try:
        cipher = AES.new(key, AES.MODE_GCM, nonce=nonce_var)
        cipher.update(metadata)
#         cipher.verify(tag_var)
        plaintext = cipher.decrypt_and_verify(cipher_text, tag_var)
        print(plaintext)
    except ValueError:
        print("AIYA")

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Firmware Update Tool')
    parser.add_argument("--infile", help="Path to the firmware image to protect.", required=True)
    parser.add_argument("--outfile", help="Filename for the output firmware.", required=True)
    parser.add_argument("--version", help="Version number of this firmware.", required=True)
    parser.add_argument("--message", help="Release message for this firmware.", required=True)
    args = parser.parse_args()

    protect_firmware(infile=args.infile, outfile=args.outfile, version=int(args.version), message=args.message)
    
    
